# Step1 - Data Source

## 1. 匯入模組 - Reload files & Import Modules

In [56]:
# reload file if you eaited them
import crawler104, config.search_params, async_example, threaded_async_job, jobs104
import data_lake
import importlib
importlib.reload(crawler104)
importlib.reload(async_example)
importlib.reload(threaded_async_job)
importlib.reload(jobs104)
importlib.reload(config.search_params)
importlib.reload(data_lake)

# import library
from crawler104 import Crawler104
from config.search_params import get_filter_params
from data_lake import DataLake

## 2. 抓取Jobs清單 - Web Crawler

In [2]:
# custom filter params for search - for yidti
role = {'ro':'全職'}
keyword = {'keyword':"後端工程師 python"}
# area = {'area':['新北市', '台北市', '桃園市', '台中市']}
isnew = {'isnew':'三日內'}
jobexp = {'jobexp':['1年以下', '1-3年']}
# 預設
mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
order = {'order':'日期排序'}
asc = {'asc':'遞減'}
# filter_params = get_filter_params(role, keyword, area, isnew, jobexp, mode, order, asc)
filter_params = get_filter_params(role, keyword, isnew, jobexp, mode, order, asc)

# keywords for filter again
job_keywords = ('工程','資料','python','data','數據','後端')
# company_exclude_keywords = pending....

# -------------------------------------------
# 根據過濾參數開始爬蟲 & 再次根據keyword過濾jobs 
crawler = Crawler104(filter_params)
filtered_jobs = crawler.run(job_keywords)

url: https://www.104.com.tw/jobs/search/?ro=1&keyword=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB+python&isnew=3&jobexp=1%2C3&mode=l&order=16&asc=0


Loading: 100%|██████████| 100/100 [02:26<00:00,  1.46s/page]


載入100頁 | 載入2969筆資料 | 過濾剩2349筆資料 | 花費 166.11 秒


## 3. 抓取Jobs內容(異步&多線程) - Web Scraper

In [3]:
jobs_details = crawler.detail(filtered_jobs)
# output (save excel file)
user = "yidti"
crawler.export(user)

jobs:2349


Processing jobs:  80%|████████  | 1890/2349 [05:11<01:35,  4.80job/s]

Error loading https://www.104.com.tw/job/7xl2a, Error: Message: timeout: Timed out receiving message from renderer: 297.209
  (Session info: chrome-headless-shell=121.0.6167.87)
Stacktrace:
	GetHandleVerifier [0x00007FF758C35E42+3538674]
	(No symbol) [0x00007FF758854C02]
	(No symbol) [0x00007FF758705AEB]
	(No symbol) [0x00007FF7586F2FE8]
	(No symbol) [0x00007FF7586F2E9F]
	(No symbol) [0x00007FF7586F13A5]
	(No symbol) [0x00007FF7586F1E5F]
	(No symbol) [0x00007FF7586FF32E]
	(No symbol) [0x00007FF758711D83]
	(No symbol) [0x00007FF7587161DA]
	(No symbol) [0x00007FF7586F2420]
	(No symbol) [0x00007FF758711B9A]
	(No symbol) [0x00007FF75878C13E]
	(No symbol) [0x00007FF75876EE53]
	(No symbol) [0x00007FF75873F514]
	(No symbol) [0x00007FF758740631]
	GetHandleVerifier [0x00007FF758C66CAD+3738973]
	GetHandleVerifier [0x00007FF758CBC506+4089270]
	GetHandleVerifier [0x00007FF758CB4823+4057299]
	GetHandleVerifier [0x00007FF758985C49+720121]
	(No symbol) [0x00007FF75886126F]
	(No symbol) [0x00007FF7588

Processing jobs: 100%|██████████| 2349/2349 [06:43<00:00,  5.82job/s]


花費 403.34 秒
CSV文件保存成功: output/yidti_2024-02-02_1.xlsx


# Step 2 - Data Lake

In [55]:
user = "yidti"
data_Lake = DataLake()
data_Lake.upload_to_nosql(user)

df length: 2349
Cell num including null: 949
Row num including null: 867
更新2349筆, 新增0筆


# Step 3 - Data Warehouse

# Step 4 - Data Application

# 99. 測試區

In [ ]:
# custom filter params for search - simon
# role = {'ro':'全職'}
# zone = {'zone': '4,5'}
# indcat = {'indcat':'1001000000,1002000000,1012000000,1014000000'}
# exclude = {'excludeJobKeyword': '軟體,會計,文件整理,行政,百貨,資訊,品管,財務,守衛,技術員,法務,品檢,登打,輪班,無經驗,客戶'}
# mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
# order = {'order':'日期排序'}
# asc = {'asc':'遞減'}
# filter_params = get_filter_params(role, zone, indcat, exclude, mode, order, asc)

In [ ]:
# 如果你在一個 Python script 中執行這樣的程式碼，你可能需要使用 asyncio.run() 函數。
# 在異步程式碼中，如果你使用 asyncio.gather 函數收集異步任務的結果，而這些任務沒有顯式返回值，
# gather 函數將返回一個包含每個異步任務結果的列表，而這些結果通常是 None。

import asyncio

async def async_example(i):
    print(f"Start asynchronous task {i}")
    await asyncio.sleep(1)
    print(f"End asynchronous task {i}")
    return f"Result of task {i}"

# 直接在 Jupyter cell 中執行
tasks = [async_example(i) for i in range(5)]
results = await asyncio.gather(*tasks)
print(results)